## Ch25 時間序列預測

我們回顧一下時間序列分析的思想：根據序列的歷史資訊來預測未來，進行預測的前提為序列是定態的，即序列的基本特徴不發生大的改變。若時間序列是特殊的定態序列－－純隨機過程（白雜訊過程），也就是無自相關性，那麼預測也無法進行。因此，我們會對定態的、非隨機序列進行分析，以預測未來。預測的方式通常是建立預測模型，預測模型的一般表達形式為：

$$x_{t+1}=f(x_i),\quad i\le t$$

也就是說，我們位於 $t$ 時點時，是利用 $t$ 期和 $t$ 期之前的資訊對下一期的變數取值進行預測。

### 25.1 移動平均預測

若時間序列具有穩定性或規則性，那麼時間序列有可能會在未來保持過去幾期的發展態勢。時間序列分析中最簡單的預測模型，就是將過去期變數取值進行平均，將平均值作為下一期的預測值。由於在取平均數時，數值的隨機波動成分在一定程度上會被消除掉，所以得到的預測值受過去極端值的干擾減少，從而達到平滑的效果，因此這種平均數預測法被稱為移動平均預測法（Moving Average）。根據「取平均」方法之不同，可以將移動平均法分為簡單移動平均法（Simple Moving Average）、加權移動平均（Weighted Moving Average）和指數移動平均法（Exponential Moving Average）等。

#### 25.1.1 簡單移動平均

簡單移動平均是用某一時點前 $n$期的數值之簡單平均數來預測該時點的數值，以達到平滑的效果。簡單地說，我們用 $x_t,x_{t-1},x_{t-2},...,x_{t-n+1}$ 這 $n$ 個數的簡單平均數來預測，可以用數學公式表達為：

$$\hat{x}_{t+1}=\frac{x_t+x_{t-1}+x_{t-2}+...+x_{t-n+1}}{n}$$

式中各變數的涵義如下：

* $\hat{x}_{t+1}$：對下一期的預測值，或者當期的簡單移動平均數；

* $x_t$：當期隨機變數的實現值；

* $x_{t-i}$：前 $i$ 期隨機變數的實現值；

* $n$：平均值的計算中包括的過去觀察值的個數。每出現一個新觀察值，就要從移動平均中減去一個最早觀察值，再加上一個最新觀察值。

#### 25.1.2 加權移動平均

簡單移動平均對每一期的數值都賦予相同的權重，也就是說，它認為每一個時期的資料對於新的預測值或者是平滑值的影響是一樣的。但現實生活中可能並非如此，即有時候簡單移動平均對於資訊的利用不是很充分。加權移動平均法則是根據同一個時間段內不同時間的資料對預測值的影響程度，分別給予不同的權重，其計算公式為：

$$\hat{x}_{t+1}=w_0x_t+w_1x_{t-1}+...+w_nx_{t-n+1}$$

* $\hat{x}_{t+1}$：對下一期的預測值，或者當期的加權移動平均數；

* $x_t$：當期隨機變數的實現值；

* $x_{t-i}$：前 $i$ 期隨機變數的實現值；

* $w_0$：當期實現值的權重；

* $w_i$：前 $i$ 期實現值的權重，如 $w_1$ 為前一期實現值的權重；以此類推

#### 25.1.3 指數加權移動平均

指數加權移動平均兼具全期平均和移動平均所長：不捨棄過去的資料，但是隨著資料時間點的遠離，其權重逐漸遞減。指數加權移動平均法的基本思想是：考慮時間間隔對事件發展的影響，各期權重隨時間間隔的増大而呈指數衰減。其計算公式如下：

$$\hat{x}_{t+1}=\alpha x_t+\alpha(1-\alpha)x_{t-1}+\alpha(1-\alpha)^2x_{t-2}+...,$$

其中，$\alpha$ 為平滑係數，滿足 $0<\alpha<1$。經驗表明，$\alpha$ 的值介於 0.05－0.3 之間，修匀效果比較好。同理可得：

$$\hat{x}_t=\alpha x_{t-1}+\alpha(1-\alpha)x_{t-2}+\alpha(1-\alpha)^2x_{t-3}+...,$$

所以結合上述兩個式子我們可以得到：

$$\hat{x}_{t+1}=\alpha x_t+(1-\alpha)\hat{x}_t$$

* $\hat{x}_{t+1}$：對下一期的預測值，或者當期的指數加權移動平均數；

* $x_t$：當期隨機變數的實現值；

* $\hat{x}_t$：$t-1$ 期對 $t$ 期的預測值，或 $t-1$ 期的指數加權移動平均數；

* $\alpha$：為平滑係數。

從整理後的式子我們可以發現，任一期的指數加權移動平均數，都是本期實際觀察值 $x_t$ 與前一期指數加權移動平均數的加權平均。確定 $\hat{x}_t$ 的初值 $\hat{x}_1$ 通常有兩種方法：一種是取第一期的實現值為初值；另一種是最初幾期的平均值為初值。

### 25.2 ARMA 模型預測

使用移動平均，我們能夠簡單、快速地對時間序列做出預測。但是，移動平均中的參數（例如選取過去實現值的期數、加權平均的權重值、指數加權平均的平滑係數等）往往是我們主觀設定的，這種主觀設定很容易偏離實際情況，因此在實際應用中，我們很少將移動平均得到的結果直接作為變數未來取值的預測。移動平均常常用在技術分析中，作為判斷股價走勢的技術指標，我們會詳細在本書第五部份介紹其應用，此是後話，此處且按下不表。

為了更好地對時間序列做出預測，學者們發展出各種各樣的時間序列模型。這些畤間序列模型意圖刻劃時間序列背後的統計規律，並根據這些規律來對時間序列做出更為精準的預測。以這種方式進行的時間序列分析之理論建構過程大致可以分為三個階段。

時間序列分析的理論研究開展地比較早，1930 年代前後是理論模型建構的基礎階段，在此期間提出的模型研究物件多是定態的時間序列。Udny Yule 在 1927 年提出了著名的自我迴歸模型（Autoregressive Model，AR 模型）[<sup>4</sup>](#fn4)；移動平均模型（Moving Average Model，MA 模型）也隨之出現。之後學者們將 AR 模型和 MA 模型結合在一起，發展出 ARMA（Autoregressive Moving Average）模型。1970 年，當 ARMA 模型被 Box 和 Jenkins 寫進教科書[<sup>5</sup>](#fn5)時，這個模型逐漸流行起來，並被廣泛應用在學術界和實務界。與此同時，Box 和 Jenkins 也在同一本書中提出了 ARIMA（Autoregressive Integrated Moving Average）模型，該模型的處理對象為非定態的時間序列，ARIMA 模型的提出標誌著時間序列分析理論建構進入成熟階段，在此階段，時間序列模型大量地被應用在實證研究中。

不過以上模型都是假設干擾項的變異數是固定不變的，在實證研究中，學者們發現金融經濟資料很多都存在異質變異數（Heteroskedasticity）現象，因此應用傳統的模型無法獲得可靠的估計結果。時間序列分析的理論建構隨後進入完善階段，Engle 在 1982 年提出 ARCH（Autoregressive Conditional Heteroskedasticity）模型，允許條件變異數發生變動。之後，Bollerslov 延伸了 ARCH 模型，提出應用範圍廣的 GARCH（Generalized Autoregressive Conditional Heteroskedasicity）模型，ARCH 和 GARCH 模型的提出很好地解決了刻畫金融資產收益率序列波動聚集的難題。

也有學者從其他的角度來完善時間序列分析模型，例如 Granger 在 1987 年提出了其整合（Cointegration）理論，將時間序列分析對象拓展到多變數的場景。

本章將重點介紹時間序列分析基礎模型，AR 模型、MA 模型和 ARMA 模型。在後面的章節中，我們會陸續介紹 ARCH 模型、GARCH 模型、共整合理論及其應用。

<span id="fn4"><sup>4</sup> Yule, G.Udny. "On a method of investigating periodicities in disturbed series, with special refrence to Wolfer's sunspot numbers." Philosophical Transaction of the Royal Society of London. Series A, Containing Papers of a Mathematical or Physical Character (1927): 267-298.</span>

<span id="fn5"><sup>5</sup> Box, George EP, and Gwilym M. Jenkins. "Time series analysis: forecasting and control."</span>

#### 25.2.1 自我迴歸模型

AR 模型是一個線性模型，將時間序列變數當期的實現值作為被解譯變數、過去期的歷史資料當作解譯變數，因此被稱作自我迴歸模型。$p$ 階自我迴歸模型（Auto Regressive Model，AR(p)）的一般表示式為：

$$x_t=\phi_0+\phi_1x_{t-1}+\phi_2x_{t-2}+...+\phi_px_{t-p}+\varepsilon_t$$

其中 $\{\varepsilon_t\}$ 是一個白雜訊序列，即滿足：

$$\mathrm{E}(\varepsilon_t)=0;\mathrm{Var}(\varepsilon_t)=\sigma_{\varepsilon}^2;\mathrm{E}(\varepsilon_t\varepsilon_s)=0,\forall s\ne t$$

且解譯變數 $x_s$ 與殘差項 $\varepsilon_t$ 無相關性，即 $\mathrm{E}(x_s\varepsilon_t)=0,\forall s<t$。為了研究 AR 模型旳統計性質，我們先舉個簡單的例子，假設定態的時間序列 $x_t$ 可以用 AR(2) 模型來刻畫：

$$x_t=\phi_0+\phi_1x_{t-1}+\phi_2x_{t-2}+\varepsilon_t,\quad |\phi_1|<1,|\phi_2|<1$$

因為定態的時間序列之均值不隨時間發生改變，所以：

$$\mu=\mathrm{E}(x_t)=\phi_0+\phi_1\mathrm{E}(x_{t-1})+\phi_2\mathrm{E}(x_{t-2})+\mathrm{E}(\varepsilon_t)$$

即：

$$\mu=\phi_0+\phi_1\mu+\phi_2\mu+0\mu=\frac{\phi_0}{1-\phi_1-\phi_2}$$

然後我們將式子左右兩邊分別將去均值 $\mu$ 可得：

$$
\begin{split}
x_t-\mu&=\phi_0+\phi_1(x_{t-1}-\mu)+\phi_2(x_{t-2}-\mu)+(\phi_1+\phi_2-1)\mu+\varepsilon_t\\
&=\phi_1(x_{t-1}-\mu)+\phi_2(x_{t-2}-\mu)+\varepsilon_t
\end{split}\label{eq:25.5}\tag{25.5}
$$

將式 ($\ref{eq:25.5}$) 左右兩邊分別乘以 $(x_{t-1}-\mu)$ 再除以變異數 $\gamma_0$ 之後，可以得到下式：

$$\frac{\gamma_1}{\gamma_0}=\phi_1\frac{\gamma_0}{\gamma_0}+\phi_2\frac{\gamma_1}{\gamma_0}$$

即

$$\rho_1=\phi_1+\phi_2\rho_1$$

可以解得：

$$\rho_1=\frac{\phi_1}{1-\phi_2}$$

用同樣的方式乘以 $(x_{t-2}-\mu)$ 可得：

$$\rho_2=\phi_1\rho_1+\phi_2$$

這樣可以計算出：

$$
\begin{split}
\rho_1&=\frac{\phi_1}{1-\phi_2}\rho_2\\
&=\phi_1\rho_1+\phi_2
\end{split}
$$

式 ($\ref{eq:25.5}$) 左右兩邊同時乘以 $(x_{t-k}-\mu),\forall k\ge 3$，可得 3 階以上（包含 3 階）的自相關係數 $\rho_k=\phi_1\rho_{k-1}+\phi_2\rho_{k-2}$，可以看出符合 AR 模型的時間序列之自相關係數會隨著階數的增加而減小，但是很多階數之後仍不等於 0，會呈現出所謂拖尾的現象。

現在我們將分析拓展至 AR(p) 模型，如果時間序列是定態的，可得：

$$\mu=\frac{\phi_0}{1-\phi_1-\phi_2-...-\phi_p}$$

將 (${\ref{eq:25.3}}$) 左右兩邊減去均值$\mu$，可得：

$$x_t-\mu=\phi_1(x_{t-1}-\mu)+\phi_2(x_{t-2}-\mu)+...+\phi_p(x_{t-p}-\mu)+\varepsilon_t\label{eq:25.6}\tag{25.6}$$

將式 ($\ref{eq:25.6}$) 左右兩邊分別乘以 $(x_t-\mu)$、$(x_{t-1}-\mu)$、..、並除以變異數 $\gamma_0$ 可得：

$$
\begin{split}
1&=\phi_1\rho_1+\phi_2\rho_2+...+\phi_p\rho_p\\
\rho_1&=\phi_1+\phi_2\rho_1+\phi_3\rho_2+...+\phi_p\rho_{p-1}\\
\rho_2&=\phi_1\rho_1+\phi_2+\phi_3\rho_1+...+\phi_p\rho_{p-2}\\
&\vdots\\
\rho_p&=\phi_1\rho_{p-1}+\phi_2\rho_{p-2}+\phi_3\rho_{p-3}+...+\phi_p
\end{split}
$$

根據這些線性關係式，我們可以解得 $\rho_1,\rho_2,...\rho_p$，對於大於 $p$ 階的自相關係數 $\rho_k$，也有：

$$\rho_k=\phi_1\rho_{k-1}+\phi_2\rho_{k-2}+...+\phi_p\rho_{k-p}$$

因此，符合 AR(p) 模型的定態時間序列，其自相關係數在 $p$ 階之後依然可能不
為 0。

#### 25.2.2 移動平均模型

MA(q) 模型認為因變數序列 $x_t$ 與隨機誤差項（Random Error）的當期值 $\varepsilon_t$ 及 $q$ 期落後值 $\varepsilon_{t-1},\varepsilon_{t-2},...\varepsilon_{t-q},$ 有關、而且是隨機誤差項的加權平均，因此被稱作移動平均模型。一個 $q$ 階移動平均模型 MA(q) 可以用數學表達為：

$$x_t=\mu+\varepsilon_t+\theta_1\varepsilon_{t-1}+\theta_2\varepsilon_{t-2}+...+\theta_q\varepsilon_{t-q}$$

其中 $\{\varepsilon_t\}$ 是白雜訊序列，滿足：

$$\mathrm{E}(\varepsilon_t)=0;\mathrm{Var}(\varepsilon_t)=\sigma_{\varepsilon}^2;\mathrm{E}(\varepsilon_t\varepsilon_s)=0,\forall s\ne t$$

由於 MA(q) 僅僅是白雜訊過程的線性組合，因此有：

$$
\begin{split}
\mathrm{E}(x_t)&=\mu,\\
\mathrm{Var}(x_t)&=\gamma_0=(1+\theta_1^2+\theta_2^2+...+\theta_q^2)\sigma_\varepsilon^2,\\
\rho_l&=
\begin{cases}
1, &l=0\\
\cfrac{(\theta_1+\theta_{l+1}\theta_1+\theta_{l+2}\theta_2+...+\theta_q\theta_{q-l})}{(1+\theta_1^2+\theta_2^2+...+\theta_q^2)}, &\forall l=1,2,...,q\\
0, &\forall l>q
\end{cases}
\end{split}
$$

由上式我們可以得知 MA(q) 模型一個很重要的統計性質：MA(q) 模型自相關係數 $q$ 階截尾。所謂的 $q$ 階截尾是指，在 $q$ 階以後 MA(q) 模型的自相關係數上截止，$q+1$ 階起就等於 0（即上式 $\gamma_l=0,\forall l>q$ 所表達的內容）。考慮 AR 模型和 MA 模型自相關係數的性質，我們可以根據自相關圖，來初步判斷所研究的時間序列大致符合什麼類別的模型。

### 25.3 ARMA 模型
AR(p) 模型認時間序列中 $x_t$ 的值與過去 $p$ 期的落後值有關，MA(q) 模型則用落後 $q$期的隨機擾動項來解譯當期的 $x_t$。不過金融經濟領域中，很多變數的值既會與自己過去期的表現有關係、又受到過去隨機誤差的影響，ARMA 模型表達的就是這個思想。ARMA 模型是研究時間序列的重要方法，由 AR 模型與 MA 模型混合構成。ARMA(p,q) 模型的一般表示式為：

$$x_t=\phi_0+\phi_1x_{t-1}+\phi_2x_{t-2}+...+\phi_px_{t-p}+\varepsilon_t+\theta_1\varepsilon_{t-1}+\theta_2\varepsilon_{t-2}+...+\theta_q\varepsilon_{t-q}$$


其中 $\{\varepsilon_t\}$ 是零均值白雜訊序列，滿足：

$$\mathrm{E}(\varepsilon_t)=0;\mathrm{Var}(\varepsilon_t)=\sigma_{\varepsilon}^2;\mathrm{E}(\varepsilon_t\varepsilon_s)=0,\forall s\ne t$$

很顯然，相較於 AR(p) 和 MA(q) 模型，ARMA(p,q) 更具有普適性，AR(p) 是 $q=0$ 時的 ARMA(p,q) 模型，MA(q) 模型是當 $p=0$ 時的 ARMA(p,q) 模型。

### 25.4 ARMA 模型的建模過程

如前所述，我們分析的對象是定態非白雜訊的序列。現在我們要建構 ARMA 模型以便預測未來，建構一個 ARMA 模型通常需要 3 個步驟：

1. 序列識別

   1. 判斷我們需要建模分析的資料是否為定態序列，若非定態序列需對其進行變換處理，使其成為定態序列。
   
   2. 接著再判斷定態的序列是否白雜訊序列，若為白雜訊序則列建模結束（白雜訊過程無法建構 ARMA 模型）；若為非白雜訊序列，則進行下一步。

2. 模型識別與估計：決定 $p$ 和 $q$ 的值，選出相對最優的模型架構。透過時間序列的自相關函數 ACF 和偏自相關函數 PACF 大致決定。一般規則如表 25.1 所示：


>表25.1：模型識別表

模型     |ACF                 |PACF
:-------:|:------------------:|:-----------------:
AR(p)    |拖尾（幾何型或震蕩型）|$p$ 階截尾
MA(q)    |$q$ 階截尾           |拖尾（幾何型或震蕩型）
ARMA(p,q)|拖尾（幾何型或震蕩型）|拖尾（幾何型或震蕩型）

如果序列的 ACF 和 PACF 不是很明確的話，我們可以嘗試著建立若干個備選模型，然後根據 AIC 或是 BIC 指標來選取。AIC 是一種用於模型選取的指標，同時考慮了模型的擬合程度以及簡單性，而 BIC 則是對 AIC 的一個改進。一般而言，較小的 AIC 或者 BIC 表明模型在保持簡單的同時能夠很好地對時間序列進行擬合。因此，我們往往會選取具有最小的 AIC 或 BIC 的模型作為相對最優的模型。

3. 模型診斷：對模型殘差進行檢定，確保其是服從常態分佈的白雜訊序列。當模型的殘差是白雜訊時，說明我們已經將序列的資訊充分截取到模型中。

### 25.5 CPI 資料的 ARMA 短期預測

這一節，我們將建立一個完整的 ARMA 模型，並用其對序列進行短期的預測，以輔助讀者更好地理解 ARMA 模型。我們使用 2001 年 1 月到 2014 年 2 月的中國大陸的月度 CPI（Consumer Price Index，物價指數）資料建立 ARMA 模型。

###### 1. 進行序列識別

我們先讀取資料，對資料進行前處理，並判斷其定態性。

In [1]:
import pandas as pd

# 讀取資料
CPI=pd.read_csv('.\\PythonBook_code_data\\part4\\025\\CPI.csv',index_col='time')

# 將資料轉換成時間序列格式便於之後的處理
CPI.index=pd.to_datetime(CPI.index)

# 檢視前三筆資料
CPI.head(n=3)

,CPI
time,
2014-05-01,100.1
2014-04-01,99.7
2014-03-01,99.5


In [2]:
# 檢視後三筆資料
CPI.tail(n=3)

,CPI
time,
2001-03-01,99.4
2001-02-01,100.2
2001-01-01,101.9
